In [1]:
# import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

from tqdm.auto import tqdm, trange
#from tqdm import tqdm  
from grape import Graph
from grape import GraphVisualizer

import matplotlib as mpl
import matplotlib.pyplot as plt
#import tikzplotlib


from grape.datasets import get_all_available_graphs_dataframe
from grape.datasets import get_dataset

import time

from math import ceil
from math import log
from math import log10

import itertools as it

from grape.edge_prediction import edge_prediction_evaluation

# Ensmallen
from grape.embedders import Node2VecCBOWEnsmallen, Node2VecGloVeEnsmallen, Node2VecSkipGramEnsmallen
from grape.embedders import DeepWalkCBOWEnsmallen, DeepWalkGloVeEnsmallen, DeepWalkSkipGramEnsmallen
from grape.embedders import WalkletsSkipGramEnsmallen, WalkletsCBOWEnsmallen, WalkletsGloVeEnsmallen
from grape.embedders import FirstOrderLINEEnsmallen, SecondOrderLINEEnsmallen
#from grape.embedders import NetMFEnsmallen, GLEEEnsmallen, HOPEEnsmallen

# TensorFlow
from grape.embedders import SkipGramTensorFlow, CBOWTensorFlow
from grape.embedders import FirstOrderLINETensorFlow, SecondOrderLINETensorFlow

# KarateClub
from grape.embedders import Role2VecKarateClub, GraRepKarateClub
from grape.embedders import NMFADMMKarateClub, RandNEKarateClub

from grape.edge_prediction import DecisionTreeEdgePrediction, PerceptronEdgePrediction

from grape.edge_prediction import edge_prediction_evaluation
from grape.edge_prediction import DecisionTreeEdgePrediction, RandomForestEdgePrediction, ExtraTreesEdgePrediction, MLPEdgePrediction, GradientBoostingEdgePrediction

from grape.embedders import HyperSketching
import pandas as pd

import tensorflow
#import karateclub
from tqdm.auto import tqdm, trange

/home/ali/.local/lib/python3.9/site-packages/ensmallen/__init__.py:56: UserWarning: Ensmallen is compiled for the Intel Haswell architecture (2013).On the current machine, the flags '{'bmi2', 'popcnt', 'avx2'}' are required but '{'bmi2', 'avx2'}' are not available.
The library will use a slower but more compatible version (Intel Core2 2006).
  warnings.warn(
2024-03-25 09:26:36.144932: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 09:26:36.268053: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 09:26:38.116842: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [39]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:

import pandas as pd
df_nodes = pd.read_csv('node_Yelp.txt', sep='\t', header=None)
df_links = pd.read_csv('link_Yelp.txt', sep='\t', header=None)
# df_links_test = pd.read_csv('link_test.txt', sep='\t', header=None)

In [20]:
df_links[df_links['link_type']==2]

,source,destination,link_type,link_label
27224784,9,7690,2,1
27224785,9,2571,2,1
27224786,9,11,2,1
27224787,9,12,2,1
27224788,9,13,2,1
...,...,...,...,...
29879092,80804,2034,2,2
29879093,80804,10740,2,1
29879094,80804,504,2,2
29879095,80804,3065,2,1


In [17]:
df_nodes.rename(columns={0: "node_id"}, inplace=True)
df_nodes.rename(columns={1: "node_name"}, inplace=True)
df_nodes.rename(columns={2: "node_type"}, inplace=True)


# link columns
df_links.rename(columns={0: "source"}, inplace=True)
df_links.rename(columns={1: "destination"}, inplace=True)
df_links.rename(columns={2: "link_type"}, inplace=True)
df_links.rename(columns={3: "link_label"}, inplace=True)

# load it into a graph
Yelp_train = Graph.from_pd(
    edges_df=df_links,
    nodes_df=df_nodes,
    node_name_column="node_id",
    node_type_column="node_type",
    edge_src_column="source",
    edge_dst_column="destination",
    #edge_weight_column="weight",
    edge_type_column="link_type",
    node_types_separator="|",
    directed=False,
    name="Yelp_train",
)

In [18]:
Yelp_train

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>Yelp_train</h2><p class="paragraph">The undirected graph Yelp_train has 82.47K heterogeneous nodes and 9.47M heterogeneous edges. The graph is connected, that is, it is composed of a single connected component that includes all nodes and edges. The RAM requirements for the nodes and edges data structures are 6.11MB and 76.43MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 1, the maximum node degree is 34.71K, the mode degree is 3, the mean degree is 229.71 and the node degree median is 70.</p><p class="paragraph">The nodes with the highest degree centrality are 50 (degree 34.71K and node type 3), 117 (degree 31.59K and node type 3), 878 (degree 21.74K and node type 3), 301 (degree 20.14K and node type 3) and 665 (degree 17.57K and node type 3).</p><h3 style="margin: 1em 0 0 0;">Node types</h3><p class="paragraph">The graph has 4 node types, which are 3 (74.94K nodes, 90.88%), 0 (7.47K nodes, 9.06%), 1 (39 nodes, 0.05%) and 2 (9 nodes, 0.01%). The RAM requirement for the node types data structure is 11.88MB.</p><h3 style="margin: 1em 0 0 0;">Edge types</h3><p class="paragraph">The graph has 4 edge types, which are 3 (13.60M edges, 71.82%), 2 (5.31M edges, 28.02%), 0 (14.95K edges, 0.08%) and 1 (14.95K edges, 0.08%). The RAM requirement for the edge types data structure is 75.77MB.</p><h3 style="margin: 1em 0 0 0;">Topological Oddities</h3><p class="paragraph">A topological oddity is a set of nodes in the graph that <i>may be derived</i> by an error during the generation of the edge list of the graph and, depending on the task, could bias the results of topology-based models. In the following paragraph, we will describe the detected topological oddities.</p><h4 style="margin: 1em 0 0 0;">Isomorphic node groups</h4><p class="paragraph">Isomorphic node groups are nodes with exactly the same neighbours, node types, edge types and weights (if present in the graph). Nodes in such groups are topologically indistinguishable, that is swapping their ID would not change the graph topology. We have detected 468 isomorphic node groups in the graph, involving a total of 1.13K nodes (1.37%) and 22.09K edges (0.12%), with the largest one involving 14 nodes and 850 edges. The detected isomorphic node groups, sorted by decreasing size, are:</p><ol class="medium-columns"><li><p class="paragraph">Group with 10 nodes (degree 85 and node type 3): 81305, 81304, 81303, 81302, 81301 and other 5.</p></li>
<li><p class="paragraph">Group with 7 nodes (degree 66 and node type 3): 71846, 71845, 71844, 71841, 71836 and other 2.</p></li>
<li><p class="paragraph">Group with 6 nodes (degree 64 and node type 3): 29765, 29768, 29780, 29778, 29788 and another one.</p></li>
<li><p class="paragraph">Group with 2 nodes (degree 147 and node type 3): 29770 and 29786.</p></li>
<li><p class="paragraph">Group with 14 nodes (degree 20 and node type 3): 34151, 34153, 34154, 34155, 34157 and other 9.</p></li>
<li><p class="paragraph">Group 

In [22]:
Node2VecSkipGram_Yelp_embedding = Node2VecSkipGram_Yelp.get_node_embedding_from_index(0)
Node2VecSkipGram_Yelp_embedding

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.707694,1.478940,0.077618,-3.657446,0.667228,1.382514,-0.889173,-2.195393,-0.168531,4.950686,...,-4.397766,1.765082,2.946609,-2.818650,-3.481653,-1.749817,-3.807460,-0.680798,-1.995230,-4.026602
1,-3.375600,0.719657,-1.863186,-3.915933,-3.519414,-3.511348,-0.331401,-0.069526,0.592969,-0.570931,...,0.020117,-0.564673,0.973255,-2.431516,-1.016219,1.410223,-2.578147,-1.548782,1.873383,-3.847881
2,0.237702,0.402386,1.012342,-3.669838,-1.363970,-1.014821,-1.979410,-3.147158,-0.116591,1.177106,...,-2.249483,3.782581,2.486035,0.833293,-2.715833,1.554817,0.327274,-1.163064,1.252510,-1.537128
3,-1.450648,-0.405541,0.636292,-2.954892,-0.121378,-2.192563,-1.077423,-2.995571,-0.268403,-0.093126,...,-1.928459,0.951938,2.832783,-0.710867,-1.686754,2.065902,-0.114227,-0.922944,1.305124,-2.086753
4,-0.474226,-0.013691,0.750897,-2.981917,0.048992,-1.778518,-1.086209,-2.269073,-0.602556,-0.366552,...,-2.209035,1.822190,2.641982,-0.868760,-2.439041,1.998378,-0.680240,-1.355971,0.501628,-1.356142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82460,-4.539089,4.249571,-2.459678,-2.694751,-0.263001,-3.849282,0.754269,-2.692149,-0.354195,2.840832,...,-7.524245,-4.744409,0.226438,-2.387600,0.819849,2.294630,2.322194,7.571267,-4.068089,2.673447
82461,0.115938,0.863128,0.928620,-2.196582,0.666936,-0.205820,-1.319739,-0.695615,-0.336867,-0.848972,...,-0.087447,0.430980,2.720256,-0.707142,-1.975509,0.542658,-1.348766,-0.214859,-1.082267,-0.827351
82462,-2.264774,0.614542,0.937541,-2.390380,0.934990,-2.150993,0.540251,-0.952523,-1.830953,-0.222881,...,0.439618,0.478124,1.254696,-1.830756,-2.747873,-0.381523,0.664920,-0.734455,-0.890337,-1.613967
82463,-1.950749,0.102490,2.296928,-1.474363,0.853859,0.626780,-0.175972,-0.080423,1.080943,1.075890,...,-0.466797,1.763888,5.436245,-1.053244,-2.595227,0.560390,-0.239218,1.564119,-0.654742,-2.206050


In [37]:
def get_embedding(graph, params):


    allNames = sorted(params)
    combinations = list(it.product(*(params[Name] for Name in allNames)))

    for index, param in enumerate(combinations) :
    
        # Embedding generation
        start = time.time()
        embedding=Node2VecSkipGramEnsmallen(
                    embedding_size=50,
                    walk_length=128,
                    iterations=10,
                    window_size=5,
                    return_weight=param[0],
                    explore_weight=param[1], 
                    change_node_type_weight=param[2],
                    change_edge_type_weight=param[2],
        ).fit_transform(graph)
        end = time.time()
        print(f'Computed {param} in {end - start} s')
        embedding = embedding.get_node_embedding_from_index(0)
        with open(f'{Yelp}_{param}.txt', 'w') as file:
            for index, row in embedding.iterrows():
                line = f"{index}\t{' '.join(map(str, row.values))}\n"
                file.write(line)

In [ ]:
params={'p':[2**p for p in [-3,-2]],'q':[2**q for q in [-3,-2]],'s':[1]}
get_embedding(Yelp_train,params)